## Few-Shot-Learning


### Divisão dos Dados

Cria um dicionário dos dados (class map): {label_classe: [lista de  imagens dessa classe]} e divide os dados entre treino e teste

**Estrutura do Omniglot**


omniglot-py/ <br>
└── images_background/  (ou images_evaluation/)<br>
    ├── Alphabet_of_the_Magi/<br>
    │   ├── character01/<br>
    │   │   ├── 0709_01.png<br>
    │   │   ├── 0709_02.png<br>
    │   │   └── ... (20 imagens no total)<br>
    │   └── character02/<br>
    │       └── ...<br>
    ├── Angelic/<br>
    │   └── ...<br>
    └── ... (mais 30 alfabetos)<br>

In [4]:
import torchvision as tv
from torchvision import transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

omniglot_train = tv.datasets.Omniglot(
    root="../data",
    background=True, #Conjunto de treino
    download=True,
    transform=transform 
)


omniglot_test = tv.datasets.Omniglot(
    root="../data",
    background=False, #Conjunto de teste
    download=True,
    transform=transform
)

100.0%
100.0%


O Omniglot é dividido em background e evaluation. O conjunto background contém os alfabetos para treinar o seu modelo a "aprender a aprender". O conjunto evaluation contém alfabetos completamente novos, que o modelo nunca viu, e é usado para testar se ele consegue generalizar para novas classes com poucos exemplos.

### Episódios

Em vez de treinar o modelo para ser um especialista em um conjunto fixo de classes, nós o treinamos para ser um generalista em aprender.

Seria dar ao modelo uma série de "mini testes surpresa" (os episódios). Cada teste é uma pequena tarefa de classificação.

**Anatomia de um Episódio**

Em um cenário N-way K-shot (onde N = 5 e K = 1):

1. Amostragem: Primeiro, escolhemos aleatoriamente N classes do nosso enorme conjunto de dados de treinamento. 
2. Support Set: Para cada uma dessas N classes, damos ao modelo K exemplos. Este é o "material de estudo" para este teste específico. O modelo deve olhar para essas "N times K" imagens e aprender a distinguir as N classes.
3. Query Set: Em seguida, pegamos outras imagens (que não estavam no material de estudo) daquelas mesmas N classes. Estas são as "perguntas do teste".
4. Avaliação e Aprendizado: O modelo tenta classificar as imagens do Query Set. Calculamos o quão bem ele se saiu (loss). Usamos essa nota para ajustar os pesos do modelo através do backpropagation.


Ao repetir esse processo milhares de vezes, com milhares de combinações diferentes de classes, o modelo não está memorizando "o que é o caractere A do alfabeto Grego". Em vez disso, ele está aprendendo uma estratégia para, dado qualquer conjunto de N novas classes com K exemplos cada, descobrir as características que as diferenciam.

In [1]:
import torch, random
import numpy as np


def create_episode(classes, n_way, k_shot, k_query):
    episode_classes_idx = random.sample(list(classes.keys()), n_way)
    
    support_set = []
    support_labels = []
    query_set = []
    query_labels = []
    
    
    #* Montar o Support e o Query set para cada classe selecionada
    for i, class_idx in enumerate(episode_classes_idx):
        images_of_class = classes[class_idx] 
        
        #* Sortear imagens dessa classe
        selected_images = random.sample(images_of_class, k_shot + k_query)
        
        #* Dividir as imagens sorteadas nos dois conjuntos
        support_images = selected_images[:k_shot]
        query_images = selected_images[k_shot:]
        
        support_set.extend(support_images)
        query_set.extend(query_images)
        
        #* Criar as labels RELATIVAS ao episódio
        support_labels.extend([i] * k_shot)
        query_labels.extend([i] * k_query)


    #* Embaralhar os conjuntos
    s_indices = np.random.permutation(len(support_set))
    q_indices = np.random.permutation(len(query_set))
    
    #* Reordenar usando os índices embaralhados
    support_set = torch.stack(support_set)[s_indices]
    support_labels = torch.tensor(support_labels)[s_indices]
    query_set = torch.stack(query_set)[q_indices]
    query_labels = torch.tensor(query_labels)[q_indices]
    
    #*Retorna os 4 tensores para serem usados pelo modelo
    return support_set, support_labels, query_set, query_labels
    


### Redes Prototípicas

A Analogia: O Crítico de Arte Novato

Imagine que seu modelo é um crítico de arte novato. Cada "episódio" é um novo desafio para ele.

O Desafio (Episódio): Levamos o crítico a uma galeria com 5 artistas que ele nunca viu antes (N_WAY = 5).

O Material de Estudo (support_set): Mostramos a ele apenas uma pintura de cada um desses 5 artistas (K_SHOT = 1). A tarefa dele é estudar essas 5 pinturas e formar uma ideia central do "estilo" de cada artista.

A Ideia Central (Protótipo): Depois de olhar a pintura de um artista, o crítico cria uma "representação mental" do estilo daquele artista. Essa representação é o protótipo. Ele faz isso para os 5 artistas.

O Teste (query_set): Agora, mostramos ao crítico uma nova pintura (que ele não usou para estudo) e perguntamos: "Qual dos 5 artistas pintou esta obra?".

A Resposta (Classificação): O crítico compara a nova pintura com as 5 "representações mentais" (protótipos) que ele criou. A que for mais parecida, ele supõe que seja do mesmo artista.

O Aprendizado (Loss & Optimizer): Se ele errar, nós o corrigimos. Essa correção o ajuda a refinar sua capacidade de extrair a "essência" de uma pintura, para que da próxima vez ele forme representações mentais melhores.

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm


#* Transforma uma imagem em um vetor de características
class EmbeddingNet(nn.Module):
    def __init__(self, embedding_dim=64):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 3 * 3, embedding_dim)
        )
    
    def forward(self, x):
        x = self.convnet(x)
        x = self.fc(x)
        return x

A rede é composta por três blocos de camadas convolucionais (Conv2d), cada um seguido por normalização em lote (BatchNorm2d), função de ativação ReLU e uma camada de pooling (MaxPool2d). O objetivo dessa sequência é extrair representações compactas e discriminativas das imagens, reduzindo gradualmente sua dimensão espacial e aumentando a profundidade das características aprendidas.

In [ ]:
#* Preparar dados de treino

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

omniglot_train = tv.datasets.Omniglot(
    root="../data",
    background=True,
    download=True,
    transform=transform
)



In [ ]:
#* Agrupar imagens por classe, facilitando a amostragem

train_class_map = {}
for img, label in omniglot_train:
    if label not in train_class_map:
        train_class_map[label] = []
    train_class_map[label].append(img)
    

In [7]:
#* Configurações de treinamento
N_WAY = 5
K_SHOT = 1
K_QUERY = 15
NUM_EPS = 1000
EMBEDDING_DIM = 64


#* Inicializando modelo e otimizador
model = EmbeddingNet(embedding_dim=EMBEDDING_DIM)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#* Loop de treinamento
print("Training...")

for episode in tqdm(range(NUM_EPS)):
    #* Pegando um novo episódio 
    support_x, support_y, query_x, query_y  = create_episode(train_class_map, N_WAY, K_SHOT, K_QUERY)
    
    #* Zerar gradientes do otimizador
    optimizer.zero_grad()
    
    #* Passar imagens de suporte e de query (teste) 
    support_embeddings = model(support_x) # Gera as representações "mentais"
    query_embeddings = model(query_x)
    
    #* Calcular o protótipo de cada classe
    prototypes = []
    for i in range(N_WAY):
        class_embeddings = support_embeddings[support_y == i]
        prototypes.append(class_embeddings.mean(dim=0))
    prototypes = torch.stack(prototypes)
    
    #* Calcular a distancia de cada imagem de teste para cada prorótipo
    distances = torch.cdist(query_embeddings, prototypes, p=2).pow(2) # Euclideana ao quadrado
    
    #* Calcular a perda do modelo
    log_p_y = F.log_softmax(-distances, dim=1)
    loss = F.nll_loss(log_p_y, query_y)
    
    #*Realiza a correção
    loss.backward()
    optimizer.step()

print("Train is over")

torch.save(model.state_dict, "prototypical_net.pth")
    

Training...


100%|██████████| 1000/1000 [01:19<00:00, 12.62it/s]

Conclued


In [ ]:
#* Carregando o conjunto de teste

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])


omniglot_test = tv.datasets.Omniglot(
    root="../data",
    background=False, 
    download=True,
    transform=transform
)

test_class_map = {}
for img, label in omniglot_test:
    if label not in test_class_map:
        test_class_map[label] = []
    test_class_map[label].append(img)

In [ ]:
#* Avaliando o modelo

model.load_state_dict(torch.load("prototypical_net.pth"))
model.eval()